In [2]:
import pandas as pd
import requests

df = pd.read_csv("OECD,DF_BLI,+all.csv")
df = df.drop(["Unnamed: 5", "Unnamed: 7", "Unnamed: 9","Unnamed: 11", "Unnamed: 13", "Unnamed: 15", "Unnamed: 17", "Unnamed: 19", "Unnamed: 21"], axis = 1)
print(df["INDICATOR"].unique())
dataframes_separados = {}

for valor, grupo in df.groupby('INDICATOR'):
    dataframes_separados[valor] = grupo.copy()

#JOBS
df2 = dataframes_separados['JE_LMIS']
df3 = dataframes_separados['JE_EMPL']
df4 = dataframes_separados['JE_LTUR']
df5 = dataframes_separados['JE_PEARN']

#HO
df6 = dataframes_separados['HO_BASE']
df7 = dataframes_separados['HO_HISH']
df8 = dataframes_separados['HO_NUMR']

#PF
df9 = dataframes_separados['PS_FSAFEN']
df10 = dataframes_separados['PS_REPH']

#Community
df11 = dataframes_separados['SC_SNTWS']

#ES
df12 = dataframes_separados['ES_EDUA']
df13 = dataframes_separados['ES_STCS']
df14 = dataframes_separados['ES_EDUEX']

#EQ
df15 = dataframes_separados['EQ_WATER']
df16 = dataframes_separados['EQ_AIRP']

#CG
df17 = dataframes_separados['CG_VOTO']
df18 = dataframes_separados['CG_SENG']

#HS
df19 = dataframes_separados['HS_SFRH']
df20 = dataframes_separados['HS_LEB']

#WL
df21 = dataframes_separados['WL_EWLH']
df22 = dataframes_separados['WL_TNOW']

#SW
df23 = dataframes_separados['SW_LIFS']

#INCOME
df24 = dataframes_separados['IW_HADI']
df25 = dataframes_separados['IW_HNFW']



['JE_LMIS' 'HO_BASE' 'PS_FSAFEN' 'HO_HISH' 'JE_EMPL' 'JE_LTUR' 'SC_SNTWS'
 'ES_EDUA' 'EQ_WATER' 'CG_VOTO' 'HS_SFRH' 'WL_EWLH' 'CG_SENG' 'ES_STCS'
 'SW_LIFS' 'HO_NUMR' 'IW_HADI' 'IW_HNFW' 'JE_PEARN' 'ES_EDUEX' 'EQ_AIRP'
 'HS_LEB' 'PS_REPH' 'WL_TNOW']


In [3]:
#JOBS
from sklearn.preprocessing import MinMaxScaler
from functools import reduce

for df in [df2, df3, df4, df5, df11, df24, df25]:
    df.columns = df.columns.str.strip()
    df['OBS_VALUE'] = df['OBS_VALUE'].fillna(0)

scaler = MinMaxScaler()
for df in [df3, df5, df11, df24, df25]:
    df['OBS_VALUE'] = scaler.fit_transform(df[['OBS_VALUE']])

for df in [df2, df4,]:
    df['OBS_VALUE'] = (1 - scaler.fit_transform(df[['OBS_VALUE']]))

df2_grouped = df2.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'JE_LMIS'})
df3_grouped = df3.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'JE_EMPL'})
df4_grouped = df4.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'JE_LTUR'})
df5_grouped = df5.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'JE_PEARN'})

df_JOBS = [df2_grouped, df3_grouped, df4_grouped, df5_grouped]
JE_Merge = reduce(lambda left, right: pd.merge(left, right, on='LOCATION', how='outer'), df_JOBS)

JE_Merge.fillna(0, inplace=True)

JE_Merge = JE_Merge.sort_values(by='LOCATION')

JE_Merge['INDEX'] = JE_Merge[['JE_LMIS', 'JE_EMPL', 'JE_LTUR', 'JE_PEARN']].mean(axis=1)

JE_Merge = JE_Merge.sort_values(by='INDEX', ascending = False)

JE_Merge


,LOCATION,JE_LMIS,JE_EMPL,JE_LTUR,JE_PEARN,INDEX
20,ISL,1.000000,0.882716,0.967949,0.964185,0.953712
29,NLD,0.927536,0.888889,0.955128,0.801287,0.893210
40,USA,0.845411,0.685185,0.977564,1.000000,0.877040
10,DEU,0.980676,0.864198,0.942308,0.705673,0.873214
26,LUX,0.942029,0.685185,0.916667,0.933449,0.869332
30,NOR,0.913043,0.833333,0.956731,0.743952,0.861765
11,DNK,0.830918,0.814815,0.955128,0.793800,0.848665
0,AUS,0.898551,0.796296,0.950321,0.733155,0.844581
1,AUT,0.937198,0.777778,0.935897,0.694142,0.836254
4,CAN,0.864734,0.740741,0.975962,0.735713,0.829288


In [4]:
#INCOME

#Not all countries appear since the df are incomplete, you can prove this statement with the following code: 
# print("df24:", df24['LOCATION'].nunique()) 
# print("df25:", df25['LOCATION'].nunique()) 
# This will show that df24 has 36 and df25 has 30 countries


df24_grouped = df24.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'IW_HADI'})
df25_grouped = df25.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'IW_HNFW'})


df_INCOME = [df24_grouped, df25_grouped]
IW_MERGE = reduce(lambda left, right: pd.merge(left, right, on='LOCATION', how='outer'), df_INCOME)

IW_MERGE.fillna(0, inplace=True)

IW_MERGE = IW_MERGE.sort_values(by='LOCATION')

IW_MERGE['INDEX'] = IW_MERGE[['IW_HADI', 'IW_HNFW']].mean(axis=1)


IW_MERGE = IW_MERGE.sort_values(by='INDEX', ascending = False)

IW_MERGE


,LOCATION,IW_HADI,IW_HNFW,INDEX
22,LUX,0.847545,1.000000,0.923772
35,USA,1.000000,0.702220,0.851110
27,NZL,0.710039,0.504593,0.607316
0,AUS,0.671985,0.521539,0.596762
14,GBR,0.567127,0.516496,0.541812
3,CAN,0.599943,0.462916,0.531429
2,BEL,0.611017,0.427375,0.519196
8,DEU,0.708771,0.261130,0.484950
26,NOR,0.712909,0.219410,0.466159
1,AUT,0.661652,0.267302,0.464477


In [5]:
SC_MERGE = df11.groupby('LOCATION', as_index=False)['OBS_VALUE'].mean().rename(columns={'OBS_VALUE': 'SC_SNTWS'})

SC_MERGE = SC_MERGE.sort_values(by='LOCATION')

SC_MERGE['INDEX'] = SC_MERGE['SC_SNTWS']

SC_MERGE = SC_MERGE.sort_values(by='INDEX', ascending = False)

SC_MERGE


,LOCATION,SC_SNTWS,INDEX
20,ISL,0.942308,0.942308
14,FIN,0.884615,0.884615
9,CZE,0.869231,0.869231
19,IRL,0.853846,0.853846
30,NOR,0.846154,0.846154
21,ISR,0.823077,0.823077
36,SVK,0.815385,0.815385
37,SVN,0.807692,0.807692
11,DNK,0.807692,0.807692
31,NZL,0.800000,0.800000


In [6]:
import plotly.graph_objects as go
import pandas as pd

merged_df = pd.merge(IW_MERGE[['LOCATION', 'INDEX']], JE_Merge[['LOCATION', 'INDEX']],
                     on='LOCATION', suffixes=('_IW', '_JE'))

merged_df = merged_df.sort_values('LOCATION')

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=merged_df['LOCATION'], y=merged_df['INDEX_IW'], mode='lines+markers', name='IW_MERGE'))
fig1.add_trace(go.Scatter(x=merged_df['LOCATION'], y=merged_df['INDEX_JE'], mode='lines+markers', name='JE_Merge'))
fig1.update_layout(title='Line Chart: INDEX by LOCATION', xaxis_title='Country Code (LOCATION)', yaxis_title='INDEX')

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=merged_df['LOCATION'], y=merged_df['INDEX_IW'], name='IW_MERGE'))
fig2.add_trace(go.Bar(x=merged_df['LOCATION'], y=merged_df['INDEX_JE'], name='JE_Merge'))
fig2.update_layout(barmode='group', title='Bar Chart: INDEX by LOCATION', xaxis_title='Country Code (LOCATION)', yaxis_title='INDEX')

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=merged_df['INDEX_IW'], y=merged_df['INDEX_JE'], mode='markers+text',
                          text=merged_df['LOCATION'], textposition='top center',
                          name='Country Comparison'))
fig3.update_layout(title='Scatter Plot: INDEX IW_MERGE vs JE_Merge by Country',
                   xaxis_title='IW_MERGE INDEX',
                   yaxis_title='JE_Merge INDEX')

fig1.show()
fig2.show()
fig3.show()


In [7]:
import plotly.express as px
from scipy.stats import linregress

# Calculate linear regression
slope, intercept, r_value, p_value, std_err = linregress(merged_df['INDEX_IW'], merged_df['INDEX_JE'])

# Print correlation results
print(f"Pearson correlation coefficient (r): {r_value:.4f}")
print(f"P-value: {p_value:.4g}")
print(f"Regression equation: INDEX_JE = {slope:.4f} * INDEX_IW + {intercept:.4f}")

# Add predicted values to DataFrame
merged_df['Predicted_JE'] = slope * merged_df['INDEX_IW'] + intercept

# Plot scatter with regression line
fig = px.scatter(merged_df, x='INDEX_IW', y='INDEX_JE', text='LOCATION',
                 title='Linear Correlation: INDEX IW_MERGE vs JE_Merge',
                 labels={'INDEX_IW': 'IW_MERGE INDEX', 'INDEX_JE': 'JE_Merge INDEX'})

# Add regression line
fig.add_scatter(x=merged_df['INDEX_IW'], y=merged_df['Predicted_JE'], mode='lines', name='Regression Line')

# Update layout
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=True)

fig.show()


Pearson correlation coefficient (r): 0.7190
P-value: 5.353e-07
Regression equation: INDEX_JE = 0.6820 * INDEX_IW + 0.4468


In [9]:
je_index = JE_Merge[['LOCATION', 'INDEX']].rename(columns={'INDEX': 'JOBS_INDEX'})
iw_index = IW_MERGE[['LOCATION', 'INDEX']].rename(columns={'INDEX': 'INCOME_INDEX'})
sc_index = SC_MERGE[['LOCATION', 'INDEX']].rename(columns={'INDEX': 'COMMUNITY_INDEX'})

dfs_to_merge = [je_index, iw_index, sc_index]
FINAL_MERGE = reduce(lambda left, right: pd.merge(left, right, on='LOCATION', how='outer'), dfs_to_merge)

FINAL_MERGE = FINAL_MERGE.fillna(0)

FINAL_MERGE['OVERALL_INDEX'] = FINAL_MERGE[['JOBS_INDEX', 'INCOME_INDEX', 'COMMUNITY_INDEX']].mean(axis=1)
FINAL_MERGE = FINAL_MERGE.sort_values(by='OVERALL_INDEX', ascending=False)

FINAL_MERGE


,LOCATION,JOBS_INDEX,INCOME_INDEX,COMMUNITY_INDEX,OVERALL_INDEX
40,USA,0.877040,0.851110,0.769231,0.832460
26,LUX,0.869332,0.923772,0.661538,0.818214
31,NZL,0.805530,0.607316,0.800000,0.737615
0,AUS,0.844581,0.596762,0.761538,0.734294
30,NOR,0.861765,0.466159,0.846154,0.724693
4,CAN,0.829288,0.531429,0.738462,0.699726
29,NLD,0.893210,0.404947,0.784615,0.694257
16,GBR,0.816672,0.541812,0.723077,0.693853
14,FIN,0.808150,0.376082,0.884615,0.689616
19,IRL,0.798913,0.409842,0.853846,0.687534
